# BVS Files Maker
`VERSION: 1.1`: Incluye lectura de `CABECERA_CONTRATO`
## Indice
> * `CABECERA_CONTRATO.txt`
> * `TARJETA_EQUIPO.csv`
> * `LINEAS_CONTRATO.csv`

## Carga de librerías

In [1]:
import pandas as pd
import numpy as np

## Lectura CABECERA_CONTRATO

In [17]:
# Lectura del archivo CABECERA_CONTRATO previamente completado por el CMDB ADMIN.
#ccr: cabecera_contrato_raw_df
ccr_df = pd.DataFrame(pd.read_excel('CABECERA_CONTRATO.xlsx', skiprows=1 ,encoding='ISO-8859-1'))
# ccr_df = pd.DataFrame(pd.read_excel('CABECERA_CONTRATO_RAW.xlsx', skiprows=1 ,encoding='ISO-8859-1'))

In [18]:
# Generación de nombres de columnas
 # cc_cabecera: es un requerimiento del SAP, se traduce como un DF con dos headers
cc_cabecera = ["ContractID", "CustomerCode", "Status", "ContractType", "Renewal", "StartDate", "EndDate", "Description",
            "ServiceType", "U_ServType", "U_Provider", "U_ServiceName"]
cc_columnas_definitivas = ["ContractID", "CstmrCode", "Status", "CntrcType", "Renewal", "StartDate", "EndDate", "Descriptio", "SrvcType", "U_ServType", "U_Provider"]


In [19]:
# Agregado de un segundo Header (Asi lo requiere SAP...). En este caso ambos Headers son idénticos

# ccr_df.columns = pd.MultiIndex.from_tuples(
#     zip(cabecera, df_tarjeta_equipo.columns))

In [20]:
# Eliminación de NaN's
ccr_df.replace(np.nan, "", inplace=True)

In [21]:
ccr_df

,ContractID,CstmrCode,Status,CntrcType,Renewal,StartDate,EndDate,Descriptio,SrvcType,U_ServType,U_Provider,U_ServiceName,U_OV,U_StatusVendor
0,0,C30500010084,scs_Approved,ct_SerialNumber,tNO,2021-01-01,2021-12-31,,bst_Regular,,CISCO,SMARTNET,123qwe,ACTIVE
1,1,C30500010084,scs_Approved,ct_SerialNumber,tNO,2021-01-01,2021-12-31,,bst_Regular,,BVS,BCARE,456asdf,ACTIVE


TODO: 
* Implementar las columas y cabecera definitivo
* Implementar el archivo .CSV


## Generación TARJETA_EQUIPO

NOTA: Un DF de esta tarjeta se utiliza para replicarlo por `ContractID` definido en CABECERA_CONTRATO

TODO: 
* Construir función que arme un DF que replique la TARJETA_EQUIPO (con sus contrato asociado) en un único DF que luego se llamará LINEAS_CONTRATO

#### Carga cruda de datos en datagrama y generación de columnas adicionales

In [6]:
# Lectura del quote Cisco (unicamente). Se descartan las 27 primeras líneas del header ("skiprows").
ter_df = pd.DataFrame(pd.read_excel('quote.xlsx', skiprows=26,encoding='ISO-8859-1'))

In [7]:
# Agregado de columnas faltantes
ter_df["insID"] = ter_df.index+1 #indice

# Estas variables se completan al ejecutar funcion de generación de DF LINEAS_CONTRATO
ter_df["customer"] = "" # Tomar feature de CABECERA_CONTRATO
ter_df["status"] = "" # Tomar feature de CABECERA_CONTRATO


In [ ]:
# Reemplazo de index por insID
# df.set_index("insID", inplace=True)

In [8]:
# Tamaño del datagrama
ter_df.shape

(2663, 68)

#### Limpieza de datos y generación de datagrama definitivo

In [9]:
# Reemplazo de np.nan's por " "
ter_df.replace(np.nan,"",inplace=True)

In [10]:
ter_df.head(3)

,Product Number,Product Description,Last Date of Support,PAK/Serial Number,Instance Number,Major/Minor,Parent Instance Number,Line Type,Host ID/Mac ID,Service Level,...,Smart Account / Virtual Account,Initial Term (Months),Billing Frequency,Automatically Renews For,Subscription ID,Reference Serial Number,Reference Instance Number,insID,customer,status
0,AIR-CT5508-50-K9,^5508 Series Controller for up to 50 APs,1690761600000000000,FCW1728L01U,1526561268,Major,1526561268,NEW,,SNT,...,CABLEVISION S.A.-DEFAULT,,,,,,,1,,
1,PI-MSE-PRMO-INSRT,"Insert, Packout - PI-MSE",,,1526561310,Minor,1526561268,NEW,,SNT,...,CABLEVISION S.A.-DEFAULT,,,,,,,2,,
2,SWC5500K9-73,Cisco Unified Wireless Controller SW Release 7.3,,,1526561320,Minor,1526561268,NEW,,SNT,...,CABLEVISION S.A.-DEFAULT,,,,,,,3,,


In [11]:
# Generación de nombres de columnas
 # filtro_columnas: son las columnas que nos interesan del DF fuente y se utilizarán para filtrar el mismo DF
 # columnas_definitivas: son los nombres con los que deben llamarse en df_tarjeta_equipo
 # cabecera: es un requerimiento del SAP, se traduce como un DF con dos headers
filtro_columnas = ["insID", "customer", "Instance Number", "PAK/Serial Number", "Product Number", "Product Description", "End Customer Address Line 1","End Customer Zip/Postal Code" ,"End Customer City", "End Customer Country", "status"]
te_columnas_definitivas = ["insID","customer", "manufSN", "internalSN", "itemCode", "itemName", "street", "zip", "city", "county", "status"]
te_cabecera = ["EquipmentCardNum", "CustomerCode", "ManufacturerSerialNum", "InternalSerialNum", "ItemCode", "ItemDescription","Street", "ZipCode", "City", "County", "StatusOfSerialNumber"]

In [12]:
# Generación de DF tarjeta_equipo "tipo" (tet_df) con las columnas de interés
te_df = pd.DataFrame(ter_df[filtro_columnas])

# DF final con los nombres de columnas actualizados según "columnas_definitivas"
te_df.columns = te_columnas_definitivas



In [14]:
#"te tipo" ya terminado y listo para utilizar en funcion de armado de DF completo
te_df.head()

,insID,customer,manufSN,internalSN,itemCode,itemName,street,zip,city,county,status
0,1,,1526561268,FCW1728L01U,AIR-CT5508-50-K9,^5508 Series Controller for up to 50 APs,GRAL HORNOS 690,1272,BUENOS AIRES,AR,
1,2,,1526561310,,PI-MSE-PRMO-INSRT,"Insert, Packout - PI-MSE",GRAL HORNOS 690,1272,BUENOS AIRES,AR,
2,3,,1526561320,,SWC5500K9-73,Cisco Unified Wireless Controller SW Release 7.3,GRAL HORNOS 690,1272,BUENOS AIRES,AR,
3,4,,1526561285,,AIR-PWR-CORD-AR,AIR Line Cord Argentina,GRAL HORNOS 690,1272,BUENOS AIRES,AR,
4,5,,1526561303,,LIC-CT5508-BASE,Base Software License,GRAL HORNOS 690,1272,BUENOS AIRES,AR,


In [16]:
te_df.shape

(2663, 11)

In [ ]:
# Agregado de un segundo Header (Asi lo requiere SAP...). En este caso ambos Headers son idénticos
tet_df.columns = pd.MultiIndex.from_tuples(
    zip(te_cabecera, tet_df.columns))

In [ ]:
# Listado de ejemplo
tet_df.head(3)

#### Generación de archivo de salida.txt

In [ ]:
# *** EN PROCESO DE TESTING ***
# *** POR EL MOMENTO NO ESTA DESARROLLADA LA SALIDA EN .TXT ***

# numpy_array = df_tarjeta_equipo.to_numpy()
# np.savetxt("test_tarjeta_equipo.txt", numpy_array, fmt = "%s")


In [ ]:
# Salida en formato xls
# tet_df.to_excel("1.test_tarjeta_equipo.xls", index = False)

In [ ]:
# Salida en formato csv
tet_df.to_csv("1.test_tarjeta_equipo.csv", index = False)

## Generación LINEAS_CONTRATO

In [ ]:
# import pandas as pd
# import numpy as np

#### Carga cruda de datos en datagrama y generación de columnas adicionales

In [ ]:
# Lectura del quote Cisco (unicamente)
lcr_df = pd.DataFrame(pd.read_excel('quote.xlsx', skiprows=26,encoding='ISO-8859-1'))

In [ ]:
# TBModify: Features para tomar nombre desde teclado
u_vendor_contract = "" # Tomarlo de CABECERA_CONTRATO "U_Provider"
u_ov = "" # Tomarlo de CABECERA_CONTRATO "U_OV" (lo incluimos por fuera del diseño original)
u_status_vendor = "" # Tomarlo de CABECERA_CONTRATO "U_StatusVendor" (lo incluimos por fuera del diseño original)

# Agregado de columnas faltantes
lcr_df["LineNum"] = lcr_df.index+1
lcr_df["U_VendorContract"] =  u_vendor_contract # Tomar feature de CABECERA_CONTRATO "U_Provider"
lcr_df["U_OV"] = u_ov # Tomar feature de CABECERA_CONTRATO "U_OV"
lcr_df["U_StatusVendor"] = u_status_vendor # Tomar feature de CABECERA_CONTRATO "U_StatusVendor"

# Tamaño del datagrama
lcr_df.shape

#### Limpieza de datos y generación de datagrama definitivo

In [ ]:
# Reemplazo de np.nan's por " "
lcr_df.replace(np.nan,"",inplace=True)

lcr_df.head(3)

In [ ]:
# Generación de nombres de columnas
 # filtro_columnas: son las columnas requeridas del DF fuente
 # columnas_definitivas: son los nombres con los que deben llamarse en df_tarjeta_equipo
filtro_columnas = ["Parent Instance Number", "LineNum", "Product Number", "Product Description", "Instance Number", "PAK/Serial Number", "Start Date","End Date" ,"U_StatusVendor", "Service Level Description", "U_OV", "End Customer Name", "U_VendorContract"]
lc_columnas_definitivas = ["ParentKey", "LineNum", "itemCode", "itemName", "ManufSN", "InternalSN", "StartDate", "EndDate", "U_StatusVendor", "U_ServSKU", "U_OV", "U_EndCustomer", "U_VendorContract"]

In [ ]:
# Generación de DF tipo (df_tarjeta_equipo) con las columnas de interés
lct_df = pd.DataFrame(lcr_df[filtro_columnas])

# DF final con los nombres de columnas actualizados según "columnas_definitivas"
lct_df.columns = lc_columnas_definitivas



# Listado de ejemplo
lct_df.head(3)

In [ ]:
# Agregado de un segundo Header (Asi lo requiere SAP...). En este caso ambos Headers son idénticos
lct_df.columns = pd.MultiIndex.from_tuples(
    zip(columnas_definitivas, lct_df.columns))

In [ ]:
#### Generación de archivo de salida.txt

# df_lineas_contrato.to_excel("2.test_lineas_contrato.xls", index = False)
lct_df.to_csv("2.test_lineas_contrato.csv", index = False)